In [1]:
import sys
import os
sys.path.insert(0, os.path.abspath(os.path.join(os.path.dirname("./Untitled.ipynb"), '..')))
import numpy as np
from src.bopf.bopf import BagOfPatternFeature

In [2]:
bopf = BagOfPatternFeature()

In [3]:
%%time
path = "D:/tesis/UCRArchive_2018/Car/Car_TRAIN.tsv"
bopf.load_dataset(path, fmt="file")

FileNotFoundError: [Errno 2] No such file or directory: 'D:/tesis/UCRArchive_2018/Car/Car_TRAIN.tsv'

In [4]:
%%time
bopf.cumsum()

Wall time: 47.8 ms


In [5]:
%%time
bopf.bop(5, 0.25)

Wall time: 1.12 s


In [6]:
%%time
bopf.adjust_label_set()

Wall time: 977 µs


In [7]:
%%time
bopf.anova()

Wall time: 571 ms


In [8]:
%%time
bopf.anova_sort()

Wall time: 0 ns


In [9]:
%%time
bopf.sort_trim_arr()

Wall time: 183 ms


In [10]:
%%time
bopf.crossVL()

Wall time: 89.8 ms


In [11]:
bopf.best_idx, bopf.best_score

(20, 0.85)

In [12]:
%%time
crossL2 = bopf.crossVL2()

Wall time: 194 ms


In [13]:
bopf.best2_idx, bopf.best2_score

(22, 0.6333333333333333)

In [15]:
bop_feature = bopf.crossL
bop_fea_num = bopf.best_idx
cv_acc = bopf.best_score
bop_feature_index = bopf.sort_index[:bop_fea_num]
bop_feature2 = bopf.crossL2
bop_fea_num2 = bopf.best2_idx
cv_acc2 = bopf.best2_score
bop_feature_index2 = bopf.sort_index[:bop_fea_num2]

In [16]:
bopf_test = BagOfPatternFeature()
path_test = "D:/tesis/UCRArchive_2018/Car/Car_TEST.tsv"
bopf_test.load_dataset(path_test, fmt="file")

In [17]:
bopf_test.cumsum()

In [18]:
%%time
bopf_test.bop(7, 0.25)

Wall time: 1.22 s


In [19]:
def sort_trim_arr(train_bop, sort_index, m, n):
    train_bop_sort = np.zeros((n+1) * m)
    idx = 0
    for j in range(n):
        k = sort_index[j]
        for i in range(m):
            train_bop_sort[idx] = train_bop[i + k * m]
            idx  += 1
    return train_bop_sort

In [20]:
test_bop_sort = sort_trim_arr(bopf_test.train_bop, bop_feature_index, bopf_test.m, bop_fea_num)

In [21]:
test_bop_sort.size

1260

In [78]:
def classify(test_bop, centroids, tlabel, mt, c, fea_num):
    res = np.zeros(mt, dtype=int)
    for i in range(mt):
        rmin = np.inf
        label = -1
        for j in range(c):
            r = 0.0
            pm = 0
            pv = 0
            for k in range(fea_num):
                pm += test_bop[i + (k)*mt]
                pv += centroids[j + (k)*c]
                d = pm - pv
                r += d*d
            
#             pm = test_bop[i]
#             pv = centroids[j]
            
            
#             for k in range(fea_num):
#                 d = pm - pv
#                 pm += test_bop[i + (k+1)*mt]
#                 pv += centroids[j+ (k+1)*c]
#                 r += d*d
            if r < rmin:
                rmin = r
                label = tlabel[j]
        res[i] = label

    return res

def classify2(test_bop, tf_idfs, tlabel, mt, c, fea_num):
    res = np.zeros(mt, dtype=int)
    for i in range(mt):
        rmin = np.inf
        label = -1
        for j in range(c):
            r1 = 0.0
            r2 = 0.0
            r3 = 0.0
            pm = 0
            pv = 0
            for k in range(fea_num):
                pm += test_bop[i + k*mt]
                pv += tf_idfs[j + (k)*c]
                d = pm
                if d > 0:
                    d = 1 + np.log10(d)
                r1 += d * pv
                r2 += d*d
                r3 += pv * pv
            r = r1 * r1 / (r2 * r3)
            if r > rmax:
                rmax = r
                label = tlabel[j]
        res[i] = label
    return res

In [23]:
predicted_label = classify(test_bop_sort, bop_feature, bopf.tlabel, bopf_test.m, bopf.c, bop_fea_num)

In [24]:
real_label = np.array(bopf_test.labels)

In [25]:
count = 0
for i in range(len(real_label)):
    if predicted_label[i] == real_label[i]:
        count += 1

print(count / len(real_label))

0.23333333333333334


In [26]:
predicted_label

array([2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

In [27]:
real_label

array([4, 3, 2, 3, 4, 3, 4, 4, 2, 4, 1, 1, 1, 3, 4, 3, 3, 4, 2, 4, 2, 2,
       3, 2, 3, 2, 3, 3, 1, 2, 4, 1, 4, 1, 4, 3, 1, 3, 1, 3, 2, 2, 3, 2,
       2, 2, 3, 3, 1, 1, 4, 3, 1, 1, 2, 1, 1, 3, 3, 4])

In [2]:
wd_arr = [3, 4, 5, 6, 7]
wd_num = 5
step = 0.05
wl_arr = (np.arange(int(1/step))+1)*step
wl_num = len(wl_arr)

In [3]:
bopf = BagOfPatternFeature()
path = "D:/tesis/UCRArchive_2018/Car/Car_TRAIN.tsv"
bopf.load_dataset(path, fmt="file")
bopf.cumsum()

In [5]:
%%time
bop_features = []
bop_fea_num = []
bop_cv_acc = []
bop_feature_index = []

bop_features2 = []
bop_fea_num2 = []
bop_cv_acc2 = []
bop_feature_index2 = []

bop_wd = []
bop_wl = []
for i in range(wd_num):
    wd = wd_arr[i]
    for j in range(wl_num):
        wl = wl_arr[j]
        bopf.bop(wd, wl)
        bopf.adjust_label_set()
        bopf.anova()
        bopf.anova_sort()
        bopf.sort_trim_arr()
        bopf.crossVL()
        
        bop_features.append(bopf.crossL[:bopf.c*bopf.best_idx])
        bop_fea_num.append(bopf.best_idx)
        bop_cv_acc.append(bopf.best_score)
        bop_feature_index.append(bopf.sort_index[:bopf.best_idx])
        
        bopf.crossVL2()
        bop_features2.append(bopf.crossL2[:bopf.c*bopf.best2_idx])
        bop_fea_num2.append(bopf.best2_idx)
        bop_cv_acc2.append(bopf.best2_score)
        bop_feature_index2.append(bopf.sort_index[:bopf.best2_idx])
        
        bop_wd.append(wd)
        bop_wl.append(wl)
        print("cross_vl_results for wl:", wl, ", wd:", wd)
        print(">>> centroid => best_fea_num:", bopf.best_idx, ", best_acc: ", bopf.best_score)
        print(">>> tf-idf => best_fea_num:", bopf.best2_idx, ", best_acc: ", bopf.best2_score)

TOTAL DE SEGMENTOS VACIOS:  0
cross_vl_results for wl: 0.05 , wd: 3
>>> centroid => best_fea_num: 19 , best_acc:  0.6333333333333333
>>> tf-idf => best_fea_num: 19 , best_acc:  0.5833333333333334
TOTAL DE SEGMENTOS VACIOS:  0
cross_vl_results for wl: 0.1 , wd: 3
>>> centroid => best_fea_num: 13 , best_acc:  0.7
>>> tf-idf => best_fea_num: 22 , best_acc:  0.5


D:\tesis\tesis\src\bopf\bopf.py:342: RuntimeWarning: invalid value encountered in double_scalars
  r = r1*r1 / (r2*r3)


TOTAL DE SEGMENTOS VACIOS:  0
cross_vl_results for wl: 0.15000000000000002 , wd: 3
>>> centroid => best_fea_num: 23 , best_acc:  0.6833333333333333
>>> tf-idf => best_fea_num: 5 , best_acc:  0.5666666666666667
TOTAL DE SEGMENTOS VACIOS:  0
cross_vl_results for wl: 0.2 , wd: 3
>>> centroid => best_fea_num: 9 , best_acc:  0.5666666666666667
>>> tf-idf => best_fea_num: 16 , best_acc:  0.5333333333333333
TOTAL DE SEGMENTOS VACIOS:  0
cross_vl_results for wl: 0.25 , wd: 3
>>> centroid => best_fea_num: 1 , best_acc:  0.8833333333333333
>>> tf-idf => best_fea_num: 11 , best_acc:  0.45
TOTAL DE SEGMENTOS VACIOS:  0
cross_vl_results for wl: 0.30000000000000004 , wd: 3
>>> centroid => best_fea_num: 1 , best_acc:  0.38333333333333336
>>> tf-idf => best_fea_num: 12 , best_acc:  0.38333333333333336
TOTAL DE SEGMENTOS VACIOS:  0
cross_vl_results for wl: 0.35000000000000003 , wd: 3
>>> centroid => best_fea_num: 8 , best_acc:  0.5
>>> tf-idf => best_fea_num: 2 , best_acc:  0.5
TOTAL DE SEGMENTOS VACIO

TOTAL DE SEGMENTOS VACIOS:  0
cross_vl_results for wl: 0.5 , wd: 7
>>> centroid => best_fea_num: 24 , best_acc:  0.9
>>> tf-idf => best_fea_num: 10 , best_acc:  0.85
TOTAL DE SEGMENTOS VACIOS:  0
cross_vl_results for wl: 0.55 , wd: 7
>>> centroid => best_fea_num: 42 , best_acc:  0.8833333333333333
>>> tf-idf => best_fea_num: 87 , best_acc:  0.85
TOTAL DE SEGMENTOS VACIOS:  0
cross_vl_results for wl: 0.6000000000000001 , wd: 7
>>> centroid => best_fea_num: 90 , best_acc:  0.8166666666666667
>>> tf-idf => best_fea_num: 57 , best_acc:  0.8333333333333334
TOTAL DE SEGMENTOS VACIOS:  0
cross_vl_results for wl: 0.65 , wd: 7
>>> centroid => best_fea_num: 159 , best_acc:  0.8666666666666667
>>> tf-idf => best_fea_num: 98 , best_acc:  0.9333333333333333
TOTAL DE SEGMENTOS VACIOS:  0
cross_vl_results for wl: 0.7000000000000001 , wd: 7
>>> centroid => best_fea_num: 40 , best_acc:  0.8166666666666667
>>> tf-idf => best_fea_num: 43 , best_acc:  0.8666666666666667
TOTAL DE SEGMENTOS VACIOS:  0
cross

In [63]:
index = np.argsort(bop_cv_acc)[::-1]
top_n = 10

avg_acc_sum = 0
for i in range(top_n):
    avg_acc_sum += bop_cv_acc[index[i]]
    print(bop_cv_acc[index[i]], bop_wd[index[i]], bop_wl[index[i]])

avg_acc = avg_acc_sum / top_n

0.9333333333333333 7 0.2
0.9333333333333333 7 0.25
0.9166666666666666 7 0.30000000000000004
0.9 6 0.5
0.9 7 0.5
0.8833333333333333 3 0.25
0.8833333333333333 7 0.55
0.8666666666666667 7 0.35000000000000003
0.8666666666666667 6 0.30000000000000004
0.8666666666666667 6 0.25


In [64]:
avg_acc

0.8950000000000001

In [67]:
bopf_t = BagOfPatternFeature()
path_t = "D:/tesis/UCRArchive_2018/Car/Car_TEST.tsv"
bopf_t.load_dataset(path, fmt="file")
bopf_t.cumsum()

In [79]:
for i in range(top_n):
    s_index = index[i]
    wd = bop_wd[s_index]
    wl = bop_wl[s_index]
    bopf_t.bop(wd, wl)
    test_bop_sort = sort_trim_arr(bopf_t.train_bop, bop_feature_index[s_index], 
                                  bopf_t.m, bop_fea_num[s_index])
    predicted_label = classify(test_bop_sort, bop_features[s_index], bopf.tlabel, 
                               bopf_t.m, bopf.c, bop_fea_num[s_index])
    real_label = np.array(bopf_t.labels)
    count = 0
    for i in range(len(real_label)):
        if predicted_label[i] == real_label[i]:
            count += 1

    print(count / len(real_label))

0.6
0.7666666666666667
0.7166666666666667
0.7
0.75
0.38333333333333336
0.7666666666666667
0.43333333333333335
0.5333333333333333
0.6333333333333333


In [55]:
for i in range(top_n):
    s_index = index2[i]
    wd = bop_wd[s_index]
    wl = bop_wl[s_index]
    bopf_t.bop(wd, wl)
    test_bop_sort = sort_trim_arr(bopf_t.train_bop, bop_feature_index2[s_index], 
                                  bopf_t.m, bop_fea_num2[s_index])
    predicted_label = classify2(test_bop_sort, bop_features2[s_index], bopf.tlabel, 
                               bopf_t.m, bopf.c, bop_fea_num2[s_index])
    real_label = np.array(bopf_t.labels)
    count = 0
    for i in range(len(real_label)):
        if predicted_label[i] == real_label[i]:
            count += 1

    print(count / len(real_label))

1.75